# Bandgap Workflow Example
 This notebook demonstrates how to build and run a bandgap workflow for a material.
 Example of building and running a bandgap workflow for twisted MoS2 interface from specific_examples.

## Process Overview
### 1. Set up the environment and parameters.
### 2. Log in to get the API token
### 3. Load the target material.
### 4. Import workflow builder and related analyzers.
### 5. Analyze material to get parameters for the workflow configuration.
### 6. Create the workflow configuration.
### 7. Create a job with material and workflow configuration.
### 8. Submit the job to the server.
### 9. Monitor the job status and retrieve results.
### 10. Display the results.

## 1. Set up the environment and parameters

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install("mat3ra-utils")
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("api_examples")

## 2. Log in to get the API token

In [ ]:
ACCOUNT_ID = "ACCOUNT_ID"
AUTH_TOKEN = "AUTH_TOKEN"
ORGANIZATION_ID = "ORGANIZATION_ID"

import os
import sys

if sys.platform == "emscripten":
    # Only works if launched within the Platform, otherwise requires redirect to Login
    apiConfig = data_from_host.get("apiConfig")
    os.environ.update(data_from_host.get("environ", {}))
    os.environ.update(
        dict(
            ACCOUNT_ID=apiConfig.get("accountId"),
            AUTH_TOKEN=apiConfig.get("authToken"),
            ORGANIZATION_ID=apiConfig.get("organizationId") or "",
        )
    )

## 3. Create material
### 3.1. Load material from local file

In [ ]:
from utils.visualize import visualize_materials as visualize
from utils.jupyterlite import load_material_from_folder

material = load_material_from_folder("/uploads", "MoS2_twisted_interface_60_degrees.json")
visualize(material)

### 3.2. Save material to the platform

In [ ]:
from utils.settings import ENDPOINT_ARGS, ACCOUNT_ID
from utils.generic import display_JSON
from exabyte_api_client.endpoints.materials import MaterialEndpoints

OWNER_ID = os.getenv("ORGANIZATION_ID") or ACCOUNT_ID
endpoint = MaterialEndpoints(*ENDPOINT_ARGS)

RAW_CONFIG = material.to_dict()
fields = ["name", "lattice", "basis"]
CONFIG = {key: RAW_CONFIG[key] for key in fields}

saved_material = endpoint.create(CONFIG, OWNER_ID)

### 3.3. Get material id

In [ ]:
material_id = saved_material.get("_id")
print("Material IDs:", material_id)

## 5. Create workflow and set its parameters
### 5.1. Get list of applications and select one

In [ ]:
from mat3ra.standata.applications import Applications

# Get Applications list (with versions, build)
apps_list = Applications.list_all()
# returns apps_list[0] = [{"name" : "espresso", "version": "7.2", "build": "GNU"}]

app = Applications.get_by_name_first_match("espresso")
# returns name, version, build config

### 5.2. Create workflow from standard workflows and preview it

In [ ]:
from mat3ra.standata.workflows import Workflows
from mat3ra.wode.workflows import Workflow
from utils.visualize import visualize_workflow

# Search WF by name and application
workflow_config = Workflows.filter_by_application(app).get_by_name_first_match("band_gap")
workflow = Workflow.create(workflow_config)

# View workflow to understand its structure
visualize_workflow(workflow)

### 5.3. Add relaxation subworkflow

In [ ]:
workflow.add_relaxation()
visualize_workflow(workflow)
# Relaxation subworkflow is added as the first subworkflow

### 5.4. Change subworkflow details (Model subtype)

In [ ]:
from mat3ra.standata.model_tree import ModelTreeStandata

swf_0 = workflow.subworkflows[0]  # relaxation subworkflow
swf_1 = workflow.subworkflows[1]  # band structure subworkflow

# Change model subtype for relaxation subworkflow
# For preview:
subtypes = ModelTreeStandata.get_subtypes_by_model_type("dft")  # ["gga", "lda"] as enum
functionals = ModelTreeStandata.get_functionals_by_subtype("dft", "lda")  # ["pz", ...] as enum

model = ModelTreeStandata.get_model_by_parameters(
    model_type="dft",
    subtype=subtypes.LDA,
    functional=functionals.PZ,
)
swf_0.model = model
swf_1.model = model

### 5.5. Modify workflow units found in preview

In [ ]:
from mat3ra.wode.context_providers import PointsGridFormDataProvider

# Values from publication
kgrid_scf = [6, 6, 1]
kgrid_nscf = [12, 12, 1]
kgrid_relax = kgrid_scf

kgrid_context_provider = PointsGridFormDataProvider(material=material)

new_context_relax = kgrid_context_provider.get_data(dimensions=kgrid_relax)
new_context_scf = kgrid_context_provider.get_data(dimensions=kgrid_scf)
new_context_nscf = kgrid_context_provider.get_data(dimensions=kgrid_nscf)

# Get workflow's specific unit that needs to be modified
# Option 1: search is done by unit name regex across the entire workflow
unit_to_modify_relax = workflow.get_unit_by_name(name_regex="relax")
unit_to_modify_relax.context.add_context(new_context_relax)

# Option 2: search is done by unit name within a specific subworkflow
unit_to_modify_scf = workflow.subworkflows[1].get_unit_by_name(name="pw_scf")
unit_to_modify_scf.context.add_context(new_context_scf)
unit_to_modify_nscf = workflow.subworkflows[1].get_unit_by_name(name="pw_nscf")
unit_to_modify_nscf.context.add_context(new_context_nscf)

# Set the modified unit back to the workflow
# Option 1: direct set by unit object, replacing the existing one
workflow.set_unit(unit_to_modify_relax)

# Option 2: set by unit flowchart id and new unit object
workflow.set_unit(unit_flowchart_id=unit_to_modify_scf.flowchart_id, new_unit=unit_to_modify_scf)
workflow.set_unit(unit_flowchart_id=unit_to_modify_nscf.flowchart_id, new_unit=unit_to_modify_nscf)

### 5.6. Save workflow to collection

In [ ]:
from exabyte_api_client import WorkflowEndpoints
from utils.settings import ENDPOINT_ARGS, ACCOUNT_ID

workflow_endpoints = WorkflowEndpoints(*ENDPOINT_ARGS)

saved_workflow_data = workflow_endpoints.create(config=workflow.to_dict(), owner_id=ACCOUNT_ID)

## 6. Create the compute configuration
### 6.1. View available clusters and providers

In [ ]:
# N/A in minimal version

### 6.2. Create compute configuration

In [ ]:
from exabyte_api_client.endpoints.jobs import JobEndpoints

exabyte_jobs_endpoint = JobEndpoints(*ENDPOINT_ARGS)
job_config = {"ppn": 8,
              "queue": "OR8",
              "nodes": 1,
              "time_limit": "00:30:00",
              "cluster": "cluster-001"
              }
compute = exabyte_jobs_endpoint.get_compute(**job_config)

## 7. Create the job with material and workflow configuration

In [ ]:
from exabyte_api_client.endpoints.materials import MaterialEndpoints
from exabyte_api_client.endpoints.projects import ProjectEndpoints

from datetime import datetime

material_endpoints = MaterialEndpoints(*ENDPOINT_ARGS)
OWNER_ID = os.getenv("ORGANIZATION_ID") or ACCOUNT_ID

project_endpoints = ProjectEndpoints(*ENDPOINT_ARGS)
project_id = project_endpoints.list({"isDefault": True, "owner._id": OWNER_ID})[0]["_id"]

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")  # for name to be found easily

JOB_NAME = f"Band Gap {timestamp}"

In [ ]:
job_endpoints = JobEndpoints(*ENDPOINT_ARGS)

jobs = job_endpoints.create_by_ids(
    materials=material,
    workflow_id=workflow["_id"],
    project_id=project_id,
    prefix=JOB_NAME,
    owner_id=OWNER_ID,
    compute=compute
)

display_JSON(jobs)

## 8. Submit the job and monitor the status

In [ ]:
job_endpoints.submit(jobs["_id"])


In [ ]:
from utils.generic import wait_for_jobs_to_finish

job_ids = [job["_id"] for job in jobs]

wait_for_jobs_to_finish(job_endpoints, job_ids, poll_interval=60)

## 9. Retrieve results

In [ ]:
import re
from utils.generic import get_property_by_subworkflow_and_unit_indicies
from exabyte_api_client.endpoints.properties import PropertiesEndpoints

property_endpoints = PropertiesEndpoints(*ENDPOINT_ARGS)

results = []
# Iterate in general, but we have only 1 material here
for material in [saved_material]:
    job = next((job for job in jobs if job["_material"]["_id"] == material["_id"]))
    final_structure = get_property_by_subworkflow_and_unit_indicies(property_endpoints, "final_structure", job, 0, 0)[
        "data"
    ]
    pressure = get_property_by_subworkflow_and_unit_indicies(property_endpoints, "pressure", job, 0, 0)["data"]["value"]
    unit_flowchart_id = job["workflow"]["subworkflows"][1]["units"][1]["flowchartId"]
    band_gap_direct = property_endpoints.get_direct_band_gap(job["_id"], unit_flowchart_id)
    band_gap_indirect = property_endpoints.get_indirect_band_gap(job["_id"], unit_flowchart_id)

    results.append(
        {
            "material_id": material["_id"],
            "angle_deg": re.search(r"(\d+(?:\.\d+)?) degrees", material["name"]).group(1),
            "band_gap_direct": band_gap_direct,
            "band_gap_indirect": band_gap_indirect,
        }
    )

## 10. Display results

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

df = pd.DataFrame(results).dropna(subset=["band_gap_direct", "band_gap_indirect"]).sort_values("angle_deg")
display(df)

plt.figure(figsize=(5, 3.6), dpi=130)
plt.scatter(df["angle_deg"], df["band_gap_direct"], marker=">", label="K-valley bandgap (direct)")
plt.scatter(df["angle_deg"], df["band_gap_indirect"], marker="<", label="Indirect bandgap")
plt.xlabel(r"$\theta$ (°)")
plt.ylabel("Energy (eV)")
plt.xlim(-2, 62)
plt.legend(frameon=False, loc="best")
plt.tight_layout()
plt.show()